# Obtenção de rating do contribuinte

Com o objetivo de direcionar o modelo que ditá quais dívidas são melhores de recuperar, será realizado um rating que envolva apenas as variáveis do contribuinte.

In [11]:
import os
import dotenv
import zipfile
import pandas as pd
import numpy as np
from datetime import date

import boto3
from io import BytesIO
import pickle

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go

import pickle
import shutil

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

import warnings
warnings.filterwarnings("ignore")

rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

True

In [12]:
def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

def paste_intervalo(row):
    return "[" + str(row['inf']) + ", " + str(row['sup']) + ")"

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3_resource.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())

# 01) Importando dados

In [13]:
zip_file = os.path.join(dataPath, 'base_treino.zip')
z = zipfile.ZipFile(zip_file)

In [14]:
def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df

In [31]:
base_conjunta = ler_bases_exportadas('imovel_mercantil.csv')
base_notas_fiscais = ler_bases_exportadas('emissao_notas.csv')

In [32]:
base_conjunta.rename(columns={'id_contribuinte': 'id_pessoa'}, inplace=True)

In [33]:
base_conjunta[['id_pessoa', 'tipo_divida']].nunique()

id_pessoa      424210
tipo_divida         2
dtype: int64

In [34]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'deb_totais', 'deb_pagos', 'idade_divida',
       'quantidade_reparcelamento', 'da_aberto', 'endereco_existe'],
      dtype='object')

In [113]:
formatar_moeda(base_conjunta.loc[base_conjunta['da_aberto'] == 1, 'valor_tot'].sum())

'R$ 13.854.531.351,30'

# Renomeando coluna de idade da dívida

In [35]:
base_conjunta = base_conjunta.rename(columns = {'idade_divida':'anos_idade_da'})
base_conjunta

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,anos_idade_da,quantidade_reparcelamento,da_aberto,endereco_existe
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,IPTU,0.0,2048.73,762.08,1286.65,...,1,0,0,0,40.0,0.0,1.0,0,1,1
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,ISS,0.0,2515.85,0.00,2515.85,...,1,0,0,0,3.0,0.0,1.0,0,1,0
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.0,1278.97,0.00,1278.97,...,1,0,0,0,4.0,0.0,21.0,0,0,0
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,ISS,0.0,847.02,847.02,0.00,...,1,0,0,0,1.0,0.0,0.0,0,1,0
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,0.0,344.12,147.20,196.92,...,1,0,0,0,6.0,0.0,11.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575657,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,0.0,1728.13,790.85,937.28,...,0,0,0,0,19.0,0.0,4.0,0,0,1
2575658,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,ISS,0.0,240.77,240.77,0.00,...,1,0,0,0,1.0,0.0,6.0,0,0,1
2575659,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,APARTAMENTO,ATIVO,IPTU,0.0,9390.43,5113.61,4276.82,...,1,0,1,0,30.0,0.0,25.0,0,1,1
2575660,fffff192f894983c9ceed47545e621dd1,imovel,dd5538d2d38803a7,CASA,ATIVO,IPTU,0.0,16594.80,11374.98,5219.82,...,1,0,1,0,32.0,0.0,15.0,0,1,1


---

In [ ]:
cdas = base_conjunta[base_conjunta.duplicated(subset=['cda'], keep=False)]
cdas


In [106]:
dff = cdas[cdas['cda'] == '8d1f5d34b36fe9dce52281a70553db462']
dff

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_pago,valor_tot,vlr_tributo,vlr_taxa,...,edificacao,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,anos_idade_da,quantidade_reparcelamento,da_aberto
1419294,8d1f5d34b36fe9dce52281a70553db462,mercantil,08d5af2c269cf4ba,RESTAURANTES E SIMILARES,INAPTO,ISS,760.65,11200.74,11200.74,0.0,...,0,1,0,0,0,6.0,4.0,6.0,2,0
1419295,8d1f5d34b36fe9dce52281a70553db462,mercantil,08d5af2c269cf4ba,RESTAURANTES E SIMILARES,INAPTO,ISS,0.00,6139.11,6139.11,0.0,...,0,1,0,0,0,1.0,0.0,3.0,2,1


---

# 02) Realizando manipulação de dados

In [36]:
valor_tot = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_tot'].sum().to_frame().reset_index()
valor_pago = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['vlr_pago'].sum().to_frame().reset_index()

# O que está em aberto
valor_aberto_tot = base_conjunta[base_conjunta['da_aberto'] == 1].groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_tot'].sum().to_frame().reset_index()
valor_aberto_pg = base_conjunta[base_conjunta['da_aberto'] == 1].groupby(['cda', 'tipo_divida', 'id_pessoa'])['vlr_pago'].sum().to_frame().reset_index()
valor_aberto = pd.merge(valor_aberto_tot, valor_aberto_pg, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")
valor_aberto['valor_aberto'] = valor_aberto['valor_tot'] - valor_aberto['vlr_pago']
valor_aberto.drop(columns = ['valor_tot', 'vlr_pago'], inplace = True)

 

valor_tot_pago = pd.merge(valor_tot, valor_pago, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")
valor_tot_pago_aberto = pd.merge(valor_tot_pago, valor_aberto, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")

 

# O que a gente esperava receber: dif_tot_pago
valor_tot_pago_aberto['dif_tot_pago'] = valor_tot_pago_aberto['valor_tot'] - valor_tot_pago_aberto['vlr_pago']

# O quanto perdeu entre o que a gente esperava receber e o que foi efetivamente pago
valor_tot_pago_aberto['dif_tot_pago_aberto'] = valor_tot_pago_aberto['dif_tot_pago'] - valor_tot_pago_aberto['valor_aberto']

valor_tot_pago_aberto.sort_values(by = 'dif_tot_pago_aberto', ascending = True)

,cda,tipo_divida,id_pessoa,valor_tot,vlr_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto
738080,4e522ef0874bf15ad8a3e5cebc255ca6E,mercantil,e9af06d7b51acbff,2537623.57,852637.39,1684986.18,1684986.18,-2.328306e-10
648147,44dcd1a526876c4c9fcacff532cf50c4E,mercantil,99aaccff68836836,1275263.88,329686.12,945577.76,945577.76,-1.164153e-10
964794,66468115f00165ba9989bae6f4af91eeE,mercantil,5c2ef5088938c509,681432.71,96396.32,585036.39,585036.39,-1.164153e-10
43192,0497ef8b0f430cc86b3a8e268c80aedfE,mercantil,fce228640d53f298,1088253.67,95173.75,993079.92,993079.92,-1.164153e-10
1561449,a58418cfd08eafbd9ceed47545e621ddE,mercantil,ebee012a761b04e9,961326.16,399858.07,561468.09,561468.09,-1.164153e-10
...,...,...,...,...,...,...,...,...
2415771,ffffd120a8f3f2799fcacff532cf50c41,imovel,934532c6c448a899,802.85,0.00,NaN,802.85,NaN
2415772,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,26104.42,26104.42,NaN,0.00,NaN
2415774,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,1728.13,0.00,NaN,1728.13,NaN
2415775,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,240.77,0.00,NaN,240.77,NaN


In [37]:
valor_tot_pago_aberto[~valor_tot_pago_aberto['dif_tot_pago_aberto'].isna()].sort_values(by = 'dif_tot_pago_aberto')

,cda,tipo_divida,id_pessoa,valor_tot,vlr_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto
738080,4e522ef0874bf15ad8a3e5cebc255ca6E,mercantil,e9af06d7b51acbff,2537623.57,852637.39,1684986.18,1684986.18,-2.328306e-10
648147,44dcd1a526876c4c9fcacff532cf50c4E,mercantil,99aaccff68836836,1275263.88,329686.12,945577.76,945577.76,-1.164153e-10
964794,66468115f00165ba9989bae6f4af91eeE,mercantil,5c2ef5088938c509,681432.71,96396.32,585036.39,585036.39,-1.164153e-10
43192,0497ef8b0f430cc86b3a8e268c80aedfE,mercantil,fce228640d53f298,1088253.67,95173.75,993079.92,993079.92,-1.164153e-10
1561449,a58418cfd08eafbd9ceed47545e621ddE,mercantil,ebee012a761b04e9,961326.16,399858.07,561468.09,561468.09,-1.164153e-10
...,...,...,...,...,...,...,...,...
1216934,8104f268246d1f919ceed47545e621ddE,mercantil,b66a0dbc22eebc5a,15326768.32,1519768.82,817497.26,13806999.50,1.298950e+07
300829,200b26abed9c0e496b3a8e268c80aedfE,mercantil,0c38fef7c41e01ce,25663880.31,182283.98,11832898.77,25481596.33,1.364870e+07
2247592,ee27ee78f6c63e6318565e26094f06eeE,mercantil,60bcc46b84243022,21113432.77,1750182.07,2886797.53,19363250.70,1.647645e+07
273829,1d271205bb109cc89ceed47545e621ddE,mercantil,a5a62b17a2ca8a04,28440312.96,734070.01,7323732.13,27706242.95,2.038251e+07


In [38]:
num_linhas_por_cda = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['da_aberto'].nunique().to_frame().sort_values(by = 'da_aberto').reset_index()
num_linhas_por_cda =  num_linhas_por_cda.rename(columns = {'da_aberto':'num_da_aberto'})
num_linhas_por_cda['ja_reneg_essa_cda'] = 0

# As entradas que tem mais de um valor para da_aberto é pq foram renegociadas
num_linhas_por_cda.loc[(num_linhas_por_cda['num_da_aberto'] > 1 ), 'ja_reneg_essa_cda'] = 1 

valor_tot_pago_aberto = pd.merge(valor_tot_pago_aberto, num_linhas_por_cda, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')


In [39]:
df_conjunta = pd.merge(base_conjunta, num_linhas_por_cda, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')

# CDAs SEM mudanca de status
cdas_01_apar = df_conjunta[df_conjunta['num_da_aberto'] == 1]
cdas_01_apar = cdas_01_apar.sort_values(by = ['cda', 'inscricao_divida'])

# CDAs com mudanca de status
cdas_gt_01_apar = df_conjunta[df_conjunta['num_da_aberto'] > 1]
cdas_gt_01_apar = cdas_gt_01_apar.sort_values(by = ['cda', 'inscricao_divida']) # 319034 

# # CDAs com mudanca de status - ABERTO
cdas_gt_01_apar_ABERTO = cdas_gt_01_apar[cdas_gt_01_apar['da_aberto'] == 1]

reconstroi = pd.concat([cdas_01_apar, cdas_gt_01_apar_ABERTO])

reconstroi = reconstroi[['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'endereco_existe','protesto',
       'ajuizamento', 'refis', 'anos_idade_da',
       'quantidade_reparcelamento', 'da_aberto']]

reconstroi = pd.merge(reconstroi, valor_tot_pago_aberto, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')

reconstroi['perc_pago'] = np.round(reconstroi['vlr_pago']/reconstroi['valor_tot'], 5)

reconstroi

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,quantidade_reparcelamento,da_aberto,valor_tot,vlr_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,IPTU,762.08,1286.65,2021-01-01,2022-09-07,...,0,1,2048.73,0.00,2048.73,2048.73,0.000000e+00,1,0,0.00000
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,ISS,0.00,2515.85,2019-01-01,2022-12-30,...,0,1,2515.85,0.00,2515.85,2515.85,0.000000e+00,1,0,0.00000
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.00,1278.97,2001-08-03,2002-09-07,...,0,0,1278.97,0.00,NaN,1278.97,NaN,1,0,0.00000
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,ISS,847.02,0.00,2022-12-01,2023-06-15,...,0,1,847.02,0.00,847.02,847.02,0.000000e+00,1,0,0.00000
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,147.20,196.92,2010-01-01,2012-10-17,...,0,0,344.12,0.00,NaN,344.12,NaN,1,0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415774,fffe69ada60a04979ceed47545e621dd1,imovel,910e31fa556db12c,CASA,ATIVO,IPTU,15601.48,0.00,2015-07-16,2015-08-10,...,0,1,33597.69,919.49,15601.48,32678.20,1.707672e+04,2,1,0.02737
2415775,fffe75c5e436f4e69fcacff532cf50c42,mercantil,eefe16fde5758b38,FOTOCÓPIAS,INAPTO,ISS,2077.97,0.00,2018-12-11,2018-12-20,...,2,1,6173.14,578.55,2077.97,5594.59,3.516620e+03,2,1,0.09372
2415776,fffe8ed282868b7d18565e26094f06ee1,imovel,1eb4adad813da472,CASA,ATIVO,IPTU,7150.17,0.00,2015-10-08,2015-11-10,...,2,1,16098.88,696.05,7150.17,15402.83,8.252660e+03,2,1,0.04324
2415777,fffeb819f46062317c8fa7a67092eaab2,mercantil,52da39231b917f91,CARGA E DESCARGA,INAPTO,ISS,2864.02,0.00,2016-12-01,2016-12-09,...,0,1,3229.55,365.53,2864.02,2864.02,4.547474e-13,2,1,0.11318


In [40]:
# CDAs que vamos treinar
da_aberto_0 = reconstroi[reconstroi['da_aberto'] == 0]
da_aberto_0

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,quantidade_reparcelamento,da_aberto,valor_tot,vlr_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.00,1278.97,2001-08-03,2002-09-07,...,0,0,1278.97,0.00,NaN,1278.97,NaN,1,0,0.0000
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,147.20,196.92,2010-01-01,2012-10-17,...,0,0,344.12,0.00,NaN,344.12,NaN,1,0,0.0000
7,000040eda866e3d19ceed47545e621dd1,imovel,9f3bac8718dac1fa,LOJA,ATIVO,IPTU,10779.87,0.00,2015-01-13,2015-01-30,...,2,0,10779.87,5438.40,NaN,5341.47,NaN,1,0,0.5045
8,000040eda866e3d19ceed47545e621dd2,mercantil,2f8ae1b9606267b4,INSTRUTOR DE TREINAMENTOS,INAPTO,ISS,328.63,0.00,2004-07-01,2008-10-05,...,0,0,328.63,0.00,NaN,328.63,NaN,1,0,0.0000
10,0000521b64bf28c8e52281a70553db461,imovel,b5df816e1786f0a3,CASA,ATIVO,IPTU,245.50,245.33,2014-01-01,2016-05-14,...,0,0,490.83,0.00,NaN,490.83,NaN,1,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255889,ffffd120a8f3f2799fcacff532cf50c41,imovel,934532c6c448a899,CASA,ATIVO,IPTU,617.95,184.90,2013-01-01,2014-07-26,...,0,0,802.85,0.00,NaN,802.85,NaN,1,0,0.0000
2255890,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,APARTAMENTO,ATIVO,IPTU,22190.68,3913.74,2021-01-01,2022-04-07,...,0,0,26104.42,26104.42,NaN,0.00,NaN,1,0,1.0000
2255891,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,790.85,937.28,2017-01-01,2019-06-28,...,0,0,1728.13,0.00,NaN,1728.13,NaN,1,0,0.0000
2255892,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,ISS,240.77,0.00,2015-01-01,2017-10-22,...,0,0,240.77,0.00,NaN,240.77,NaN,1,0,0.0000


In [46]:
da_aberto_0.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'vlr_tributo', 'vlr_taxa', 'competencia_divida',
       'inscricao_divida', 'arrecadacao_divida', 'ajuizamento_divida',
       'edificacao', 'cpf_cnpj_existe', 'endereco_existe', 'protesto',
       'ajuizamento', 'refis', 'anos_idade_da', 'quantidade_reparcelamento',
       'da_aberto', 'valor_tot', 'vlr_pago', 'valor_aberto', 'dif_tot_pago',
       'dif_tot_pago_aberto', 'num_da_aberto', 'ja_reneg_essa_cda',
       'perc_pago'],
      dtype='object')

# 03) Criando variáveis para clusterização

In [47]:
print("Gerando variáveis para identificação dos grupos de contribuintes")

dados_pessoas = da_aberto_0[['tipo_divida', 'id_pessoa', 'cda',  
                               'cpf_cnpj_existe', 'edificacao', 'situacao', 'endereco_existe',
                               'valor_tot', 'vlr_pago', 
                               'quantidade_reparcelamento', 'ja_reneg_essa_cda',
                               'anos_idade_da']]

dados_pessoas.dropna(subset = ['id_pessoa'], inplace = True)
dados_pessoas

Gerando variáveis para identificação dos grupos de contribuintes


,tipo_divida,id_pessoa,cda,cpf_cnpj_existe,edificacao,situacao,endereco_existe,valor_tot,vlr_pago,quantidade_reparcelamento,ja_reneg_essa_cda,anos_idade_da
2,mercantil,96e8e553de69d7a4,00000b44c5ba1e669ceed47545e621dd2,1,0,INAPTO,0,1278.97,0.00,0,0,21.0
4,imovel,870c08c252b25ad1,0000331f601a73e52b46f673bf0c61251,1,1,ATIVO,1,344.12,0.00,0,0,11.0
7,imovel,9f3bac8718dac1fa,000040eda866e3d19ceed47545e621dd1,1,1,ATIVO,1,10779.87,5438.40,2,0,15.0
8,mercantil,2f8ae1b9606267b4,000040eda866e3d19ceed47545e621dd2,1,0,INAPTO,1,328.63,0.00,0,0,15.0
10,imovel,b5df816e1786f0a3,0000521b64bf28c8e52281a70553db461,1,1,ATIVO,1,490.83,0.00,0,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2255889,imovel,934532c6c448a899,ffffd120a8f3f2799fcacff532cf50c41,1,1,ATIVO,1,802.85,0.00,0,0,9.0
2255890,imovel,c6b16a1235d987d3,ffffd79d8aa612ac9fcacff532cf50c41,1,1,ATIVO,1,26104.42,26104.42,0,0,1.0
2255891,imovel,d411f54ad97d1f39,ffffd8af5760cada9fcacff532cf50c41,0,0,ATIVO,1,1728.13,0.00,0,0,4.0
2255892,mercantil,51d6043a4efa1ed1,ffffdb721b90282718565e26094f06ee2,1,0,ATIVO,1,240.77,0.00,0,0,6.0


In [54]:
reconstroi.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'vlr_tributo', 'vlr_taxa', 'competencia_divida',
       'inscricao_divida', 'arrecadacao_divida', 'ajuizamento_divida',
       'edificacao', 'cpf_cnpj_existe', 'endereco_existe', 'protesto',
       'ajuizamento', 'refis', 'anos_idade_da', 'quantidade_reparcelamento',
       'da_aberto', 'valor_tot', 'vlr_pago', 'valor_aberto', 'dif_tot_pago',
       'dif_tot_pago_aberto', 'num_da_aberto', 'ja_reneg_essa_cda',
       'perc_pago'],
      dtype='object')

# Agrupamento para mapear o comportamento do contribuinte

In [55]:
# Calculo que apresenta quantas cdas o contribuinte tem

frequencia_da_pessoa = da_aberto_0.groupby(['id_pessoa', 'tipo_divida'])['cda'].nunique().to_frame().reset_index()
total_reparcelamentos_pessoa = da_aberto_0.groupby(['id_pessoa', 'tipo_divida'])['quantidade_reparcelamento'].sum().to_frame().reset_index()
valor_total_pessoa = da_aberto_0.groupby(['id_pessoa', 'tipo_divida'])['valor_tot'].sum().to_frame().reset_index()
valor_pago_pessoa = da_aberto_0.groupby(['id_pessoa', 'tipo_divida'])['vlr_pago'].sum().to_frame().reset_index()

# valor_total_pessoa_anteriormente = da_aberto_0.groupby(['id_pessoa', 'tipo_divida'])['valor_tot_anteriormente'].sum().to_frame().reset_index()
# valor_pago_pessoa_anteriormente = da_aberto_0.groupby(['id_pessoa', 'tipo_divida'])['valor_pago_anteriormente'].sum().to_frame().reset_index()

# ja_reneg_essa_cda = da_aberto_0[da_aberto_0['ja_reneg_essa_cda'] == 1].groupby(['id_pessoa', 'tipo_divida'])['cda'].nunique().to_frame().reset_index()


In [56]:
# TRATA A VARIÁVEL EDIFICAÇÃO
edificacao = dados_pessoas[['tipo_divida',	'id_pessoa', 'edificacao']]
edificacao = edificacao.drop_duplicates()

In [57]:
# TRATA A VARIÁVEL SITUAÇÃO
situacao = dados_pessoas[['tipo_divida', 'id_pessoa', 'situacao']]
situacao = situacao.drop_duplicates()

In [58]:
# TRATA A VARIÁVEL DE DOCUMENTAÇÃO DO CONTRIBUINTE
cpf_cnpj_existe = dados_pessoas[['tipo_divida', 'id_pessoa', 'cpf_cnpj_existe']]
cpf_cnpj_existe = cpf_cnpj_existe.drop_duplicates()

In [60]:
# CRIA UM CONTEXTO PARA CADA PESSOA EM RELAÇÃO AOS DADOS

CHAVE = ['id_pessoa', 'tipo_divida']
freq_repal = pd.merge(frequencia_da_pessoa, total_reparcelamentos_pessoa, on = CHAVE, how = 'left') # id_pessoa	- tipo_divida - cda - quantidade_reparcelamento
freq_repal = freq_repal.rename(columns = {'cda':'num_dist_cda'})

valor = pd.merge(valor_total_pessoa, valor_pago_pessoa, on = CHAVE, how = 'outer') # id_pessoa	tipo_divida	valor_tot	valor_pago
notas_edif = pd.merge(base_notas_fiscais, edificacao, on = 'id_pessoa', how = 'outer') # id_pessoa	qtd_notas_2anos	tipo_divida	edificacao
situ_doc = pd.merge(situacao, cpf_cnpj_existe, on = CHAVE, how = 'outer') # tipo_divida	id_pessoa	situacao	cpf_cnpj_existe

# freq_repal_debitos = pd.merge(freq_repal, debitos, on = CHAVE, how = 'left')
valor_notas_edif = pd.merge(valor, notas_edif, on = CHAVE, how = 'left')

# freq_repal_debitos_valor_notas_edif = pd.merge(freq_repal_debitos, valor_notas_edif, on = CHAVE, how = 'left')

pessoas = pd.merge(valor_notas_edif, situ_doc, on = CHAVE, how = 'left')

In [61]:
# Substituindo por zero os valores nulos da coluna edificacao e notas fiscais
pessoas['qtd_notas_2anos'] = pessoas['qtd_notas_2anos'].fillna(0)
pessoas['edificacao'] = pessoas['edificacao'].fillna(0)

In [62]:
pessoas.columns

Index(['id_pessoa', 'tipo_divida', 'valor_tot', 'vlr_pago', 'qtd_notas_2anos',
       'edificacao', 'situacao', 'cpf_cnpj_existe'],
      dtype='object')

In [19]:
# REGRAS DE APLICAÇÃO REFERENTE A ACESSIBILIDADE DE COBRANÇA DO CONTRIBUINTE

# MERCANTIL
pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] > 0) & (pessoas['situacao'] == 'ATIVO'), 'perfil_acessivel'] = 2
pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] > 0) & (pessoas['situacao'] != 'ATIVO'), 'perfil_acessivel'] = 1
pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] == 0) & (pessoas['situacao'] == 'ATIVO'), 'perfil_acessivel'] = 1
pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] == 0) & (pessoas['situacao'] != 'ATIVO'), 'perfil_acessivel'] = 0

# IMOVEL
pessoas.loc[(pessoas['tipo_divida'] == 'imovel' ) & (pessoas['edificacao'] == 1), 'perfil_acessivel'] = 2
pessoas.loc[(pessoas['tipo_divida'] == 'imovel' ) & (pessoas['edificacao'] == 0), 'perfil_acessivel'] = 0

In [20]:
# SE O CONTRIBUINTE LEVA ZERO NO PERFIL DE COBRANÇA ELE LEVA ZERO NA SITUAÇÃO DE COBRANÇA
pessoas['situacao_cobranca'] = pessoas['perfil_acessivel'] + pessoas['cpf_cnpj_existe'] + pessoas['endereco_existe']
pessoas.loc[(pessoas['endereco_existe'] == 0), 'situacao_cobranca'] = 0
pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['perfil_acessivel'] == 0), 'situacao_cobranca'] = 0


In [21]:
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca
0,00000b449b8ad90b,mercantil,6,0,34.0,0.0,32725.94,0.00,0.0,0.0,SUSPENSO,1,0.0,0.0
1,000032f6d93a0abd,imovel,2,2,10.0,8.0,12715.63,3339.31,0.0,1.0,ATIVO,1,2.0,3.0
2,000032f6d93a0abd,mercantil,9,0,66.0,0.0,69091.42,0.00,0.0,0.0,INAPTO,1,0.0,0.0
3,00006a29d93b27bb,mercantil,1,0,2.0,0.0,1714.47,0.00,0.0,0.0,BAIXADO,1,0.0,0.0
4,00006a4fa855b308,imovel,1,1,11.0,2.0,1132.33,364.32,0.0,1.0,ATIVO,1,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,fffeb78fd42d2624,mercantil,3,0,15.0,0.0,13050.71,0.00,0.0,0.0,SUSPENSO,1,0.0,0.0
494409,fffee3cacb174a68,mercantil,1,0,1.0,1.0,486.92,486.92,0.0,0.0,SUSPENSO,1,0.0,0.0
494410,ffff460622e1e234,mercantil,2,0,4.0,0.0,692.98,0.00,0.0,0.0,INAPTO,1,0.0,0.0
494411,ffff7f9bfb240bda,mercantil,2,3,66.0,64.0,153878.16,54121.14,0.0,0.0,BAIXADO,1,0.0,0.0


In [22]:
dicionario_situacao_cobranca = {
    'situacao_cobranca': [0, 1, 2, 3],
    'class_situacao_cobranca': ['INACESSÍVEL', 
                                'POUCO ACESSÍVEL', 
                                'BEM ACESSÍVEL',
                                'MUITO ACESSÍVEL']
                                
}
dicionario_situacao_cobranca = pd.DataFrame(dicionario_situacao_cobranca)
dicionario_situacao_cobranca

,situacao_cobranca,class_situacao_cobranca
0,0,INACESSÍVEL
1,1,POUCO ACESSÍVEL
2,2,BEM ACESSÍVEL
3,3,MUITO ACESSÍVEL


In [23]:
pessoas = pd.merge(pessoas, 
         dicionario_situacao_cobranca, 
         on = "situacao_cobranca",
         how = "left")

pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca
0,00000b449b8ad90b,mercantil,6,0,34.0,0.0,32725.94,0.00,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL
1,000032f6d93a0abd,imovel,2,2,10.0,8.0,12715.63,3339.31,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL
2,000032f6d93a0abd,mercantil,9,0,66.0,0.0,69091.42,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL
3,00006a29d93b27bb,mercantil,1,0,2.0,0.0,1714.47,0.00,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL
4,00006a4fa855b308,imovel,1,1,11.0,2.0,1132.33,364.32,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,fffeb78fd42d2624,mercantil,3,0,15.0,0.0,13050.71,0.00,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL
494409,fffee3cacb174a68,mercantil,1,0,1.0,1.0,486.92,486.92,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL
494410,ffff460622e1e234,mercantil,2,0,4.0,0.0,692.98,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL
494411,ffff7f9bfb240bda,mercantil,2,3,66.0,64.0,153878.16,54121.14,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL


In [24]:
# Faz o calculo do historico de pagamento
pessoas.loc[(pessoas['deb_totais'].isna()) | (pessoas['deb_totais'] == 0) , 'deb_totais'] = 1
pessoas.loc[(pessoas['valor_tot'].isna()) | (pessoas['valor_tot'] == 0) , 'valor_tot'] = 1

pessoas['historico_pagamento_em_qtd'] = pessoas['deb_pagos'] / (pessoas['deb_totais'])
pessoas['historico_pagamento_em_valor'] = pessoas['vlr_pago'] / (pessoas['valor_tot'])
pessoas = pessoas.sort_values(by = 'historico_pagamento_em_valor', ascending = False)

## CRIANDO A PIPE DE DADOS PARA CLUSTERIZAÇÃO

In [25]:
# Filtrando contribuintes com mais de uma CDA
df_pipe_cluster = pessoas.query("num_dist_cda > 1")
    

df_pipe_cluster = df_pipe_cluster[['id_pessoa',
                                    'tipo_divida',
                                    'situacao_cobranca',
                                    'num_dist_cda',              # nova (antigo frequencia_da_pessoa)
                                    'quantidade_reparcelamento', # nova
                                    'historico_pagamento_em_qtd', 
                                    'historico_pagamento_em_valor']]
df_pipe_cluster = df_pipe_cluster.set_index(['id_pessoa', 'tipo_divida'])

In [26]:
df_pipe_cluster.head()

,,situacao_cobranca,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
id_pessoa,tipo_divida,,,,,
f1e9dde2a3053d51,imovel,3.0,4,0,1.0,1.0
196d80b1b82edbc0,mercantil,0.0,2,2,1.0,1.0
5f21929a71c17a08,imovel,3.0,4,3,1.0,1.0
5f202122440e32a5,mercantil,0.0,2,2,1.0,1.0
cb7ae840eac57e1d,mercantil,3.0,2,2,1.0,1.0


In [27]:
df_pipe_cluster.describe().T

,count,mean,std,min,25%,50%,75%,max
situacao_cobranca,390502.0,1.451516,1.382237,0.0,0.0,2.0,3.000000,3.0
num_dist_cda,390502.0,5.917990,3.490616,2.0,3.0,5.0,8.000000,112.0
quantidade_reparcelamento,390502.0,1.428738,3.608940,0.0,0.0,0.0,1.000000,221.0
historico_pagamento_em_qtd,390502.0,0.241666,0.360382,0.0,0.0,0.0,0.448276,1.0
historico_pagamento_em_valor,390502.0,0.146567,0.272039,0.0,0.0,0.0,0.166122,1.0


# 03) Clusterização dos contribuintes com algoritmo Kmeans 

In [28]:
faixa_n_clusters = [i for i in range(2,16)]
valores_inercia = []
valores_silhouette = []

for k in faixa_n_clusters:
    agrupador = KMeans(n_clusters=k, random_state=1337)
    label = agrupador.fit_predict(df_pipe_cluster)
    print(f"Treinamento do agrupador para K= {k} finalizado")
    
    media_inercia = agrupador.inertia_
    valores_inercia.append(media_inercia)
    print(f"Inércia calculada para o agrupador de K= {k}. Inércia: {media_inercia}")

    # silhouette_avg = silhouette_score(df_pipe_cluster, label)
    # valores_silhouette.append(silhouette_avg)
    # print(f"Silhouette Score calculado para o agrupador de K={k}. Score: {silhouette_avg}")

Treinamento do agrupador para K= 2 finalizado
Inércia calculada para o agrupador de K= 2. Inércia: 7027799.036579832
Treinamento do agrupador para K= 3 finalizado
Inércia calculada para o agrupador de K= 3. Inércia: 4367005.563949874
Treinamento do agrupador para K= 4 finalizado
Inércia calculada para o agrupador de K= 4. Inércia: 3472099.8395680375
Treinamento do agrupador para K= 5 finalizado
Inércia calculada para o agrupador de K= 5. Inércia: 2905367.690651757
Treinamento do agrupador para K= 6 finalizado
Inércia calculada para o agrupador de K= 6. Inércia: 2459929.5022264444
Treinamento do agrupador para K= 7 finalizado
Inércia calculada para o agrupador de K= 7. Inércia: 2104615.890514022
Treinamento do agrupador para K= 8 finalizado
Inércia calculada para o agrupador de K= 8. Inércia: 1875105.0271434607
Treinamento do agrupador para K= 9 finalizado
Inércia calculada para o agrupador de K= 9. Inércia: 1680542.3800328705
Treinamento do agrupador para K= 10 finalizado
Inércia calcu

A inércia é uma métrica usada para avaliar a qualidade dos clusters. Ela fornece uma medida de quão compactos são os clusters, ou seja, quão próximos os pontos de dados estão do centro (centróide) do seu respectivo cluster. A ideia é que clusters mais compactos tenham uma inércia menor.

Estamos procurando um ponto no gráfico onde a diminuição na inércia começa a desacelerar e a curva parece formar um cotovelo. Esse ponto é considerado um indicador do número ideal de clusters, que no nosso caso é 5

In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_clusters, y = valores_inercia))
fig.update_layout(
    title = "INDICADOR: Inercia para 5 grupos",
    xaxis_title = "Número de Clusters (K)",
    yaxis_title = "Inércia",
    font = dict(
        family = "Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [ ]:
# from sklearn.metrics import silhouette_score
# from joblib import Parallel, delayed

# def calculate_silhouette_score(k, data):
#     modelo = KMeans(n_clusters=k, random_state=1337)
#     labels = modelo.fit_predict(df_pipe_cluster)
#     return silhouette_score(df_pipe_cluster, labels)

# faixa_n_clusters = [i for i in range(2, 16)]
# valores_silhouette = []

# for k in faixa_n_clusters:
#     scores = Parallel(n_jobs=-1)(delayed(calculate_silhouette_score)(k, df_pipe_cluster) for _ in range(10))
#     media_score = sum(scores) / len(scores)
#     valores_silhouette.append(media_score)
#     print(f"Silhouette Score calculado para o agrupador de K={k}. Score: {media_score}")

O Silhouette Score é uma métrica usada para avaliar a qualidade dos clusters gerados por algoritmos de clusterização, como o K-Means. Ela fornece uma medida de quão bem os objetos em um cluster são separados uns dos outros em comparação com quão bem eles estão agrupados em seu próprio cluster em relação aos clusters vizinhos. Em outras palavras, o Silhouette Score mede a coesão intracluster e a separação intercluster.

- Um valor próximo de 1 indica que os objetos estão bem agrupados e separados, com alta coesão intracluster e boa separação entre clusters.
- Um valor próximo de 0 indica que os clusters estão sobrepostos ou que os objetos estão próximos do limite entre dois clusters.
- Um valor próximo de -1 indica que os objetos foram atribuídos a clusters errados, pois têm mais semelhanças com objetos em outros clusters do que com os do próprio cluster.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=faixa_n_clusters, y=valores_silhouette, mode='lines+markers'))
fig.update_layout(
    title="Silhouette Score em relação ao número de clusters",
    xaxis_title="Número de Clusters (K)",
    yaxis_title="Silhouette Score",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

O Erro Quadrático Médio (MSE - Mean Squared Error) é uma métrica comumente usada para avaliar a qualidade de modelos de regressão, onde a saída é uma variável contínua. O MSE mede o quão bem as previsões de um modelo se aproximam dos valores reais, calculando a média dos quadrados das diferenças entre as previsões e os valores reais. No entanto, o K-Means é um algoritmo de clusterização, que é usado para agrupar dados em clusters com base em similaridade, e não para fazer previsões em uma escala contínua.

In [30]:
# Métrica de validação e escolha ideal de k para clusterização indicando um bom número de clusters para a clusterização.
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 15, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [31]:
valor_ideal_k = optimal_number_of_clusters(valores_inercia)
print("Melhor número de Clusters (K):", valor_ideal_k)

Melhor número de Clusters (K): 5


In [32]:
# Construindo o melhor agrupador de clusteres
VALOR_K = valor_ideal_k

agrupador = KMeans(n_clusters=VALOR_K, random_state=1337)
agrupador.fit_transform(df_pipe_cluster)

# Obtendo o ponto central dos clusteres
centros = agrupador.cluster_centers_
df_centroide = pd.DataFrame(centros, columns = df_pipe_cluster.columns).round(3)
df_centroide['cluster'] = df_centroide.index

# Obtendo o label para cada pessoa
df_pipe_cluster['label_cluster'] = agrupador.labels_

In [33]:
df_pipe_cluster

,,situacao_cobranca,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,label_cluster
id_pessoa,tipo_divida,,,,,,
f1e9dde2a3053d51,imovel,3.0,4,0,1.0,1.0,0
196d80b1b82edbc0,mercantil,0.0,2,2,1.0,1.0,0
5f21929a71c17a08,imovel,3.0,4,3,1.0,1.0,0
5f202122440e32a5,mercantil,0.0,2,2,1.0,1.0,0
cb7ae840eac57e1d,mercantil,3.0,2,2,1.0,1.0,0
...,...,...,...,...,...,...,...
6dec5209fbc2f813,imovel,3.0,4,0,0.0,0.0,0
6dec36900dea378c,mercantil,0.0,7,0,0.0,0.0,4
6debc091e4bf4d84,mercantil,0.0,2,0,0.0,0.0,0


In [34]:
df_pipe_cluster.groupby('label_cluster').count()

,situacao_cobranca,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
label_cluster,,,,,
0,173567,173567,173567,173567,173567
1,69052,69052,69052,69052,69052
2,33723,33723,33723,33723,33723
3,4793,4793,4793,4793,4793
4,109367,109367,109367,109367,109367


## Centroide ( Média das variáveis agrupadas em cada cluster )

In [35]:
df_centroide

,situacao_cobranca,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,cluster
0,1.468,2.998,0.672,0.308,0.212,0
1,1.935,11.443,0.690,0.071,0.034,1
2,2.586,6.641,7.504,0.720,0.367,2
3,2.612,11.115,23.339,0.696,0.238,3
4,0.719,6.615,0.258,0.077,0.041,4


In [36]:
# Para ter uma noção de quem seria o centroide
df_primeira_divida = pessoas.query("num_dist_cda == 1")

# Imputando historico_pagamento_em_valor = 1 nos casos que passa de 1
df_primeira_divida.loc[df_primeira_divida['historico_pagamento_em_valor'] > 1, 'historico_pagamento_em_valor'] = 1
df_primeira_divida

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor
247206,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0
183548,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0
183545,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL,1.0,1.0
183502,5f1c8f8ff37e7c3e,mercantil,1,0,2.0,2.0,475.05,475.05,0.0,0.0,ATIVO,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0
183488,5f1b78f91fafb2fd,imovel,1,0,5.0,5.0,256.24,256.24,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212539,6deaad1ecbbf645a,imovel,1,0,20.0,0.0,1818.79,0.00,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.0,0.0
212526,6de7ee41718e00fe,imovel,1,0,14.0,0.0,642.70,0.00,0.0,1.0,ATIVO,0,2.0,2.0,BEM ACESSÍVEL,0.0,0.0
212545,6deb3707706b25c7,mercantil,1,0,4.0,0.0,1833.70,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0
212546,6deb4b0cba7a76c7,mercantil,1,0,1.0,0.0,321.94,0.00,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL,0.0,0.0


### Comentários 1ª Dívida

#### 39% inacessível

#### 46% completamente acessível

In [37]:
prim_div_status_sit = df_primeira_divida.groupby('situacao_cobranca')['id_pessoa'].nunique().to_frame().reset_index()
total = prim_div_status_sit['id_pessoa'].sum()
prim_div_status_sit['total'] = total
prim_div_status_sit['perc'] = np.round(prim_div_status_sit['id_pessoa']/total, 5)
prim_div_status_sit

,situacao_cobranca,id_pessoa,total,perc
0,0.0,41253,103510,0.39854
1,1.0,2255,103510,0.02179
2,2.0,11884,103510,0.11481
3,3.0,48118,103510,0.46486


In [38]:
data_percentil = {
    'inf': [0, 0.01, 0.25, 0.5, 0.75, 0.99, 1],
    'sup': [0.01, 0.25, 0.5, 0.75, 0.99, 1, 1.01]
}

# Creating a DataFrame from the dictionary
df_cut_percentil = pd.DataFrame(data_percentil)
df_cut_percentil['intervalo'] = df_cut_percentil.apply(paste_intervalo, axis = 1)

# Define the bins and labels for groups
igr_bins = [0, 0.01, 0.25, 0.5, 0.75, 0.99, 1, 1.01]
igr_labels = df_cut_percentil['intervalo']


df_primeira_divida['historico_pagamento_em_qtd_faixas'] = pd.cut(df_primeira_divida['historico_pagamento_em_qtd'],
                                bins = igr_bins,
                                labels = igr_labels,
                                right = False)


df_primeira_divida['historico_pagamento_em_valor_faixas'] = pd.cut(df_primeira_divida['historico_pagamento_em_valor'],
                                bins = igr_bins,
                                labels = igr_labels,
                                right = False)

df_hist_pg_vlr = df_primeira_divida.groupby('historico_pagamento_em_valor_faixas')['historico_pagamento_em_valor_faixas'].count().to_frame()
total = df_hist_pg_vlr['historico_pagamento_em_valor_faixas'].sum()  
df_hist_pg_vlr['total'] = total
df_hist_pg_vlr['perc'] = np.round(df_hist_pg_vlr['historico_pagamento_em_valor_faixas']/total, 4)
df_hist_pg_vlr

,historico_pagamento_em_valor_faixas,total,perc
historico_pagamento_em_valor_faixas,,,
"[0.0, 0.01)",53973,103911,0.5194
"[0.01, 0.25)",7913,103911,0.0762
"[0.25, 0.5)",8612,103911,0.0829
"[0.5, 0.75)",3434,103911,0.0330
"[0.75, 0.99)",217,103911,0.0021
"[0.99, 1.0)",0,103911,0.0000
"[1.0, 1.01)",29762,103911,0.2864


In [39]:
df_hist_pg_qtd = df_primeira_divida.groupby('historico_pagamento_em_qtd_faixas')['historico_pagamento_em_qtd_faixas'].count().to_frame()
total = df_hist_pg_qtd['historico_pagamento_em_qtd_faixas'].sum()  
df_hist_pg_qtd['total'] = total
df_hist_pg_qtd['perc'] = np.round(df_hist_pg_qtd['historico_pagamento_em_qtd_faixas']/total, 4)
df_hist_pg_qtd

,historico_pagamento_em_qtd_faixas,total,perc
historico_pagamento_em_qtd_faixas,,,
"[0.0, 0.01)",53860,103911,0.5183
"[0.01, 0.25)",1313,103911,0.0126
"[0.25, 0.5)",734,103911,0.0071
"[0.5, 0.75)",3007,103911,0.0289
"[0.75, 0.99)",15138,103911,0.1457
"[0.99, 1.0)",99,103911,0.0010
"[1.0, 1.01)",29760,103911,0.2864


Na totalidade temos +1 cluster especial que é do contribuinte que apareceu a primeira vez em DA

In [40]:
# dicionario_clusteres ={            # status       # qtd   # vlr
#     1: 'PIOR PAGADOR',          C  # 0.006	    0.113	0.045   --->  paga pouquíssimo & tá inacessível     
#     3: 'PAGADOR INTERMEDIARIO', B  # 1.999	    0.177	0.077   --->  paga pouquíssimo & tá menos acessível
#     2: 'BOM PAGADOR',           A  # 3.000 	    0.139	0.067   --->  paga pouquíssimo & tá acessível         
#     0: 'MELHOR PAGADOR',        AA # 2.954	    0.874	0.606   --->  paga bem         & tá acessível	        
#     4: 'PRIMEIRA DIVIDA'           # 1.668        0.437   0.351   --->  paga médio       & tá mais ou menos acessível
# }

# Com K = 4
#status_situacao 	num_dist_cda	quantidade_reparc	historico_pagamento_em_qtd	historico_pagamento_em_valor	cluster
#2.595	            6.761	        7.513	                0.713	                        0.361	                        0
#1.548	            9.855	        0.486	                0.073	                        0.037	                        1
#2.627	            11.095	        23.413	                0.698	                        0.239	                        2
#1.277	            3.601	        0.562	                0.251	                        0.170	                        3

# Com k = 5
#status_situacao num_dist_cda	quantidade_reparc	hist_pg_qtd	   hist_pg_valor   cluster  Num.Contrib
#0.768	         6.601	         0.259	            0.077	        0.041	        0   --- 108.634  # pg mau + nao repar + num médio de CDAs + pouquíssimo acessível [CONTRIB PESSIMO]
#1.490	         2.998	         0.672	            0.307	        0.212	        1   --- 173.614  # pg médio + nao repar + num baixo de CDAs + pouco acessível     [MÉDIO INACESSÍVEL]
#2.606	         6.629	         7.518	            0.721	        0.367	        2   --- 33.602   # pg melhor + repar bem + num médio de CDAs + mt acessível       [CONTRIB EXCEL]
#2.627	         11.115	         23.339	            0.696	        0.238	        3   --- 4.793    # pg bem + repar mt + num alto de CDAs + mt acessível            [CONTRIB NEGOC]
#2.019	         11.414	         0.693	            0.072	        0.034	        4   --- 69.859   # pg mau + nao repar + num alto de CDAs + médio acessível        [RUIM ACESSÍVEL]


#1.468	        2.998	        0.672	            0.308	        0.212	        0  ------- pg medio + nao repar + num baixo de cdas + pouco acessível              [CONTRIB RAZOAVEL]
#1.935	        11.443	        0.690	            0.071	        0.034	        1  ------- pg mau + nao repar + num alto cdas + pouco acessível                    [CONTRIB RUIM]
#2.586	        6.641	        7.504	            0.720	        0.367	        2  ------- pg melhor + repar bem + num medio de cdas + mt acessível                [CONTRIB EXCEL]
#2.612	        11.115	        23.339	            0.696	        0.238	        3  ------- pg bem + repar mt + num alto cdas + mt acessível                        [CONTRIB BOM]
#0.719	        6.615	        0.258	            0.077	        0.041	        4  ------- pg mau + nao repar + num medio de cdas + pouquíssimo acessível          [CONTRIB PESSIMO]

# Criar um dicionário com valores escalares
dicionario_clusteres = {
    'class_contribuinte': [0, 1, 2, 3, 4, 5],
    'class_contribuinte_nome': ['CONTRIB RAZOAVEL', 
                                'CONTRIB RUIM', 
                                'CONTRIB EXCEL',
                                'CONTRIB BOM', 
                                'CONTRIB PESSIMO',
                                'PRIMEIRA DIVIDA'],

    'class_contribuinte_perfil': ['CONTROLADO', 
                                'MUITA DÍVIDA', 
                                'PAGA BEM', 
                                'REPARCELADOR', 
                                'NÃO PAGA',
                                'NOVO EM DÍVIDA']
}

df_dicionario_clusteres = pd.DataFrame(dicionario_clusteres)
df_dicionario_clusteres

,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil
0,0,CONTRIB RAZOAVEL,CONTROLADO
1,1,CONTRIB RUIM,MUITA DÍVIDA
2,2,CONTRIB EXCEL,PAGA BEM
3,3,CONTRIB BOM,REPARCELADOR
4,4,CONTRIB PESSIMO,NÃO PAGA
5,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA


In [41]:
df_pipe_cluster.groupby('label_cluster').count()

,situacao_cobranca,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
label_cluster,,,,,
0,173567,173567,173567,173567,173567
1,69052,69052,69052,69052,69052
2,33723,33723,33723,33723,33723
3,4793,4793,4793,4793,4793
4,109367,109367,109367,109367,109367


In [42]:
df_centroide

,situacao_cobranca,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,cluster
0,1.468,2.998,0.672,0.308,0.212,0
1,1.935,11.443,0.690,0.071,0.034,1
2,2.586,6.641,7.504,0.720,0.367,2
3,2.612,11.115,23.339,0.696,0.238,3
4,0.719,6.615,0.258,0.077,0.041,4


In [198]:
# Salve o DataFrame em um arquivo CSV
df_dicionario_clusteres.to_csv(r'C:\Users\Consultor\Documents\bases_pesquisa\verificacao\dic_clusters.csv', index = False)  


Classificações dos contribuintes com base no melhor ao pior pagador, baseado no seu histórico e na sua situação atual.

# 04) Cria classificador de class do contribuinte

In [43]:
# Constroi o modelo que prevê qual o grupo do contribuinte

x_cluster = df_pipe_cluster.drop(columns=['label_cluster'])
y_cluster = df_pipe_cluster['label_cluster']

X_train, X_test, y_train, y_test = train_test_split(x_cluster, y_cluster, test_size=0.3, random_state=1337)

In [44]:
model_predict_contribuinte = RandomForestClassifier(random_state=1337)
model_predict_contribuinte.fit(X_train, y_train)

score_validacao = model_predict_contribuinte.score(X_test, y_test)
print("Score de validacao:", score_validacao)

Score de validacao: 0.9998719601198454


In [45]:
# Previsão da classificação para a base total de contribuintes
matriz_previsao_class = pessoas[['situacao_cobranca', 'num_dist_cda', 'quantidade_reparcelamento', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]
pessoas['class_contribuinte'] = model_predict_contribuinte.predict(matriz_previsao_class)

pessoas.loc[pessoas['num_dist_cda'] == 1, 'class_contribuinte'] = 5

In [46]:
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
247206,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0,5
467137,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,0
183548,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,5
183545,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,5
49086,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212557,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.0,0.0,0
212556,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4
212552,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,0
212551,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4


In [47]:
opa = pessoas[pessoas['id_pessoa'] == 'cc5afb41f6fbc367']
opa

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
394624,cc5afb41f6fbc367,mercantil,3,2,10.0,5.0,4204.59,505.70,230.0,0.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.500000,0.120273,0
394623,cc5afb41f6fbc367,imovel,9,0,126.0,2.0,7887.28,63.44,230.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.015873,0.008043,1


# Exemplo com hash de pessoas

In [204]:
pessoas[pessoas['id_pessoa'] == '000032f6d93a0abd'] 
#status_situacao num_dist_cda	quantidade_reparc	hist_pg_qtd	   hist_pg_valor   cluster  Num.Contrib
#0.768	         6.601	         0.259	            0.077	        0.041	        0   --- 108.634  # pg mau + nao repar + num médio de CDAs + pouquíssimo acessível [CONTRIB PESSIMO]
#1.490	         2.998	         0.672	            0.307	        0.212	        1   --- 173.614  # pg médio + nao repar + num baixo de CDAs + pouco acessível     [MÉDIO INACESSÍVEL]


,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
1,000032f6d93a0abd,imovel,2,2,10.0,8.0,12715.63,3339.31,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.8,0.262615,0
2,000032f6d93a0abd,mercantil,9,0,66.0,0.0,69091.42,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.000000,4


In [205]:
pessoas[pessoas['id_pessoa'] == 'def2e7cb1e2f6ae1'] 
#status_situacao num_dist_cda	quantidade_reparc	hist_pg_qtd	   hist_pg_valor   cluster  Num.Contrib
#1.490	         2.998	         0.672	            0.307	        0.212	        1   --- 173.614  # pg médio + nao repar + num baixo de CDAs + pouco acessível     [MÉDIO INACESSÍVEL]
#2.606	         6.629	         7.518	            0.721	        0.367	        2   --- 33.602   # pg melhor + repar bem + num médio de CDAs + mt acessível       [CONTRIB EXCEL]


,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
430200,def2e7cb1e2f6ae1,imovel,7,7,45.0,43.0,4980.41,4137.32,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.955556,0.830719,2
430201,def2e7cb1e2f6ae1,mercantil,2,0,6.0,0.0,33600.12,0.00,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL,0.000000,0.000000,0


In [206]:
# Nomeando a classificação com label de prioridade

pessoas = pd.merge(pessoas, 
         df_dicionario_clusteres, 
         on = "class_contribuinte",
         how = "left")

pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA


In [207]:
pessoas.columns

Index(['id_pessoa', 'tipo_divida', 'num_dist_cda', 'quantidade_reparcelamento',
       'deb_totais', 'deb_pagos', 'valor_tot', 'vlr_pago', 'qtd_notas_2anos',
       'edificacao', 'situacao', 'cpf_cnpj_existe', 'perfil_acessivel',
       'situacao_cobranca', 'class_situacao_cobranca',
       'historico_pagamento_em_qtd', 'historico_pagamento_em_valor',
       'class_contribuinte', 'class_contribuinte_nome',
       'class_contribuinte_perfil'],
      dtype='object')

In [208]:
# Incluindo variável 'tipo_divida'
df_classificao_contribuinte = pessoas[['id_pessoa', 'tipo_divida','class_contribuinte_nome', 'class_contribuinte_perfil']]
df_classificao_contribuinte

,id_pessoa,tipo_divida,class_contribuinte_nome,class_contribuinte_perfil
0,7fcd58cfe066e299,mercantil,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
1,f1e9dde2a3053d51,imovel,CONTRIB RAZOAVEL,CONTROLADO
2,5f225ff188d71bf6,imovel,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
3,5f21929a71c17a08,mercantil,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
4,196d80b1b82edbc0,mercantil,CONTRIB RAZOAVEL,CONTROLADO
...,...,...,...,...
494408,6dec5209fbc2f813,imovel,CONTRIB RAZOAVEL,CONTROLADO
494409,6dec36900dea378c,mercantil,CONTRIB PESSIMO,NÃO PAGA
494410,6debc091e4bf4d84,mercantil,CONTRIB RAZOAVEL,CONTROLADO
494411,6debbb4cc485f25f,mercantil,CONTRIB PESSIMO,NÃO PAGA


In [209]:
df_classificao_contribuinte.groupby('class_contribuinte_nome').count()

,id_pessoa,tipo_divida,class_contribuinte_perfil
class_contribuinte_nome,,,
CONTRIB BOM,4793,4793,4793
CONTRIB EXCEL,33730,33730,33730
CONTRIB PESSIMO,109368,109368,109368
CONTRIB RAZOAVEL,173566,173566,173566
CONTRIB RUIM,69045,69045,69045
PRIMEIRA DIVIDA,103911,103911,103911


In [210]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'deb_totais', 'deb_pagos', 'anos_idade_da',
       'quantidade_reparcelamento', 'da_aberto'],
      dtype='object')

# 05) Análise discriminante da classificação do contribuintes para o % Pago das dívidas de cada CDA

In [211]:
# Calcula variável target y
base_conjunta.loc[(base_conjunta['vlr_pago'].isna()) | (base_conjunta['vlr_pago'] == 0) , 'vlr_pago'] = 1
base_conjunta.loc[(base_conjunta['valor_tot'].isna()) | (base_conjunta['valor_tot'] == 0) , 'valor_tot'] = 1

base_conjunta['percentual_pago_cda'] = base_conjunta['vlr_pago'] / base_conjunta['valor_tot']

In [212]:
# Imputando historico_pagamento_em_valor = 1 nos casos que passa de 1

base_conjunta.loc[base_conjunta['percentual_pago_cda'] > 1, 'percentual_pago_cda'] = 1

In [213]:
df_classificao_contribuinte

,id_pessoa,tipo_divida,class_contribuinte_nome,class_contribuinte_perfil
0,7fcd58cfe066e299,mercantil,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
1,f1e9dde2a3053d51,imovel,CONTRIB RAZOAVEL,CONTROLADO
2,5f225ff188d71bf6,imovel,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
3,5f21929a71c17a08,mercantil,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
4,196d80b1b82edbc0,mercantil,CONTRIB RAZOAVEL,CONTROLADO
...,...,...,...,...
494408,6dec5209fbc2f813,imovel,CONTRIB RAZOAVEL,CONTROLADO
494409,6dec36900dea378c,mercantil,CONTRIB PESSIMO,NÃO PAGA
494410,6debc091e4bf4d84,mercantil,CONTRIB RAZOAVEL,CONTROLADO
494411,6debbb4cc485f25f,mercantil,CONTRIB PESSIMO,NÃO PAGA


In [214]:
# Separação dos dados para a análise discriminante dos grupos de contribuintes
df_analise_discriminante = pd.merge(
    left = base_conjunta, on=['id_pessoa', 'tipo_divida'], right=df_classificao_contribuinte, how='left'
)

df_analise_discriminante = df_analise_discriminante[['id_pessoa', 'tipo_divida', 'percentual_pago_cda', 'class_contribuinte_nome', 'class_contribuinte_perfil']]

In [215]:
base_conjunta

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,anos_idade_da,quantidade_reparcelamento,da_aberto,percentual_pago_cda
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,IPTU,1.0,2048.73,762.08,1286.65,...,1,0,0,0,40.0,0.0,1.0,0,1,0.000488
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,ISS,1.0,2515.85,0.00,2515.85,...,1,0,0,0,3.0,0.0,1.0,0,1,0.000397
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,1.0,1278.97,0.00,1278.97,...,1,0,0,0,4.0,0.0,21.0,0,0,0.000782
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,ISS,1.0,847.02,847.02,0.00,...,1,0,0,0,1.0,0.0,0.0,0,1,0.001181
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,1.0,344.12,147.20,196.92,...,1,0,0,0,6.0,0.0,11.0,0,0,0.002906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414893,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,1.0,1728.13,790.85,937.28,...,0,0,0,0,19.0,0.0,4.0,0,0,0.000579
2414894,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,ISS,1.0,240.77,240.77,0.00,...,1,0,0,0,1.0,0.0,6.0,0,0,0.004153
2414895,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,APARTAMENTO,ATIVO,IPTU,1.0,9390.43,5113.61,4276.82,...,1,0,1,0,30.0,0.0,25.0,0,1,0.000106
2414896,fffff192f894983c9ceed47545e621dd1,imovel,dd5538d2d38803a7,CASA,ATIVO,IPTU,1.0,16594.80,11374.98,5219.82,...,1,0,1,0,32.0,0.0,15.0,0,1,0.000060


In [216]:
df_analise_discriminante = df_analise_discriminante.set_index(['id_pessoa', 'tipo_divida'])
df_analise_discriminante

,,percentual_pago_cda,class_contribuinte_nome,class_contribuinte_perfil
id_pessoa,tipo_divida,,,
ac28642d7c82b33f,imovel,0.000488,CONTRIB RAZOAVEL,CONTROLADO
fc4b99b807fbed41,mercantil,0.000397,CONTRIB PESSIMO,NÃO PAGA
96e8e553de69d7a4,mercantil,0.000782,CONTRIB RAZOAVEL,CONTROLADO
6dbe14da38a31dc1,mercantil,0.001181,CONTRIB EXCEL,PAGA BEM
870c08c252b25ad1,imovel,0.002906,CONTRIB RUIM,MUITA DÍVIDA
...,...,...,...,...
d411f54ad97d1f39,imovel,0.000579,CONTRIB RUIM,MUITA DÍVIDA
51d6043a4efa1ed1,mercantil,0.004153,CONTRIB RAZOAVEL,CONTROLADO
9b512f210b460120,imovel,0.000106,CONTRIB EXCEL,PAGA BEM


In [217]:
df_analise_discriminante = df_analise_discriminante.reset_index()#.groupby("class_contribuinte_nome").describe().T
df_analise_discriminante

,id_pessoa,tipo_divida,percentual_pago_cda,class_contribuinte_nome,class_contribuinte_perfil
0,ac28642d7c82b33f,imovel,0.000488,CONTRIB RAZOAVEL,CONTROLADO
1,fc4b99b807fbed41,mercantil,0.000397,CONTRIB PESSIMO,NÃO PAGA
2,96e8e553de69d7a4,mercantil,0.000782,CONTRIB RAZOAVEL,CONTROLADO
3,6dbe14da38a31dc1,mercantil,0.001181,CONTRIB EXCEL,PAGA BEM
4,870c08c252b25ad1,imovel,0.002906,CONTRIB RUIM,MUITA DÍVIDA
...,...,...,...,...,...
2414893,d411f54ad97d1f39,imovel,0.000579,CONTRIB RUIM,MUITA DÍVIDA
2414894,51d6043a4efa1ed1,mercantil,0.004153,CONTRIB RAZOAVEL,CONTROLADO
2414895,9b512f210b460120,imovel,0.000106,CONTRIB EXCEL,PAGA BEM
2414896,dd5538d2d38803a7,imovel,0.000060,CONTRIB EXCEL,PAGA BEM


In [218]:
# Dummyzando a variável de classificação 

ohe = OneHotEncoder(dtype=int)

colunas_ohe = ohe.fit_transform(df_analise_discriminante[['class_contribuinte_nome']]).toarray()
df_2 = pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['class_contribuinte_nome']))
df_2 

,class_contribuinte_nome_CONTRIB BOM,class_contribuinte_nome_CONTRIB EXCEL,class_contribuinte_nome_CONTRIB PESSIMO,class_contribuinte_nome_CONTRIB RAZOAVEL,class_contribuinte_nome_CONTRIB RUIM,class_contribuinte_nome_PRIMEIRA DIVIDA
0,0,0,0,1,0,0
1,0,0,1,0,0,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,0,0,1,0
...,...,...,...,...,...,...
2414893,0,0,0,0,1,0
2414894,0,0,0,1,0,0
2414895,0,1,0,0,0,0
2414896,0,1,0,0,0,0


In [219]:
df_n_categorico = df_analise_discriminante.drop(columns=['class_contribuinte_nome'], axis=1)
df_n_categorico

,id_pessoa,tipo_divida,percentual_pago_cda,class_contribuinte_perfil
0,ac28642d7c82b33f,imovel,0.000488,CONTROLADO
1,fc4b99b807fbed41,mercantil,0.000397,NÃO PAGA
2,96e8e553de69d7a4,mercantil,0.000782,CONTROLADO
3,6dbe14da38a31dc1,mercantil,0.001181,PAGA BEM
4,870c08c252b25ad1,imovel,0.002906,MUITA DÍVIDA
...,...,...,...,...
2414893,d411f54ad97d1f39,imovel,0.000579,MUITA DÍVIDA
2414894,51d6043a4efa1ed1,mercantil,0.004153,CONTROLADO
2414895,9b512f210b460120,imovel,0.000106,PAGA BEM
2414896,dd5538d2d38803a7,imovel,0.000060,PAGA BEM


In [220]:
df_n_categorico = df_n_categorico.drop(columns=['class_contribuinte_perfil'], axis=1)
df_n_categorico

,id_pessoa,tipo_divida,percentual_pago_cda
0,ac28642d7c82b33f,imovel,0.000488
1,fc4b99b807fbed41,mercantil,0.000397
2,96e8e553de69d7a4,mercantil,0.000782
3,6dbe14da38a31dc1,mercantil,0.001181
4,870c08c252b25ad1,imovel,0.002906
...,...,...,...
2414893,d411f54ad97d1f39,imovel,0.000579
2414894,51d6043a4efa1ed1,mercantil,0.004153
2414895,9b512f210b460120,imovel,0.000106
2414896,dd5538d2d38803a7,imovel,0.000060


In [221]:
df_pipe_discriminante = pd.concat([df_n_categorico, df_2], axis=1)

In [222]:
df_pipe_discriminante = df_pipe_discriminante.set_index(['id_pessoa', 'tipo_divida'])
df_pipe_discriminante

,,percentual_pago_cda,class_contribuinte_nome_CONTRIB BOM,class_contribuinte_nome_CONTRIB EXCEL,class_contribuinte_nome_CONTRIB PESSIMO,class_contribuinte_nome_CONTRIB RAZOAVEL,class_contribuinte_nome_CONTRIB RUIM,class_contribuinte_nome_PRIMEIRA DIVIDA
id_pessoa,tipo_divida,,,,,,,
ac28642d7c82b33f,imovel,0.000488,0,0,0,1,0,0
fc4b99b807fbed41,mercantil,0.000397,0,0,1,0,0,0
96e8e553de69d7a4,mercantil,0.000782,0,0,0,1,0,0
6dbe14da38a31dc1,mercantil,0.001181,0,1,0,0,0,0
870c08c252b25ad1,imovel,0.002906,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
d411f54ad97d1f39,imovel,0.000579,0,0,0,0,1,0
51d6043a4efa1ed1,mercantil,0.004153,0,0,0,1,0,0
9b512f210b460120,imovel,0.000106,0,1,0,0,0,0


In [223]:
x_analise_discriminante = df_pipe_discriminante.drop(columns=['percentual_pago_cda'])
y_analise_discriminante = df_pipe_discriminante['percentual_pago_cda'].astype('int')

In [224]:
analise_discriminante = LinearDiscriminantAnalysis()
analise_discriminante.fit(x_analise_discriminante, y_analise_discriminante)

LinearDiscriminantAnalysis()

In [225]:
dados_analise_disc = {'variavel': analise_discriminante.feature_names_in_, 'coeficiente' : analise_discriminante.coef_[0].round(5)}
pesos_analise_disc = pd.DataFrame(dados_analise_disc).sort_values('variavel').reset_index().drop(columns=['index'])

In [226]:
pesos_analise_disc

,variavel,coeficiente
0,class_contribuinte_nome_CONTRIB BOM,0.18231
1,class_contribuinte_nome_CONTRIB EXCEL,1.52929
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79044
3,class_contribuinte_nome_CONTRIB RAZOAVEL,0.84045
4,class_contribuinte_nome_CONTRIB RUIM,-0.89564
5,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14338


In [227]:
pesos_analise_disc['class_contribuinte_nome'] = pesos_analise_disc['variavel'].str.replace('class_contribuinte_nome_', '')
pesos_analise_disc

,variavel,coeficiente,class_contribuinte_nome
0,class_contribuinte_nome_CONTRIB BOM,0.18231,CONTRIB BOM
1,class_contribuinte_nome_CONTRIB EXCEL,1.52929,CONTRIB EXCEL
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79044,CONTRIB PESSIMO
3,class_contribuinte_nome_CONTRIB RAZOAVEL,0.84045,CONTRIB RAZOAVEL
4,class_contribuinte_nome_CONTRIB RUIM,-0.89564,CONTRIB RUIM
5,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14338,PRIMEIRA DIVIDA


In [228]:
pesos_analise_disc.to_csv(r'C:\Users\Consultor\Documents\bases_pesquisa\verificacao\peso_clusters.csv', index = False) 

# 06) Salva o modelo de classificação dos contribuintes

In [229]:
def salva_modelo_serializado(nome_modelo_serializado, modelo):
    sav_best_model = open(nome_modelo_serializado, 'wb')
    pickle.dump(modelo, sav_best_model)
    sav_best_model.close()

    pathModelo = modelsPath+"\\"+os.path.join(nome_modelo_serializado)
    shutil.move(os.path.abspath(nome_modelo_serializado), pathModelo)

In [230]:
salva_modelo_serializado("classificador-contribuinte-v2.pkl", model_predict_contribuinte)

In [231]:
# Use o método str.replace() para substituir o caractere '#' por uma string vazia na coluna 'texto'
pesos_analise_disc['class_contribuinte_nome'] = pesos_analise_disc['variavel'].str.replace('class_contribuinte_nome_', '')
pesos_analise_disc 

,variavel,coeficiente,class_contribuinte_nome
0,class_contribuinte_nome_CONTRIB BOM,0.18231,CONTRIB BOM
1,class_contribuinte_nome_CONTRIB EXCEL,1.52929,CONTRIB EXCEL
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79044,CONTRIB PESSIMO
3,class_contribuinte_nome_CONTRIB RAZOAVEL,0.84045,CONTRIB RAZOAVEL
4,class_contribuinte_nome_CONTRIB RUIM,-0.89564,CONTRIB RUIM
5,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14338,PRIMEIRA DIVIDA


In [232]:
pesos_analise_disc = pd.merge(pesos_analise_disc, 
         df_dicionario_clusteres, 
         on = "class_contribuinte_nome",
         how = "left")

pesos_analise_disc  

,variavel,coeficiente,class_contribuinte_nome,class_contribuinte,class_contribuinte_perfil
0,class_contribuinte_nome_CONTRIB BOM,0.18231,CONTRIB BOM,3,REPARCELADOR
1,class_contribuinte_nome_CONTRIB EXCEL,1.52929,CONTRIB EXCEL,2,PAGA BEM
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79044,CONTRIB PESSIMO,4,NÃO PAGA
3,class_contribuinte_nome_CONTRIB RAZOAVEL,0.84045,CONTRIB RAZOAVEL,0,CONTROLADO
4,class_contribuinte_nome_CONTRIB RUIM,-0.89564,CONTRIB RUIM,1,MUITA DÍVIDA
5,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14338,PRIMEIRA DIVIDA,5,NOVO EM DÍVIDA


In [233]:
freq_classecontrib = pessoas.groupby(['class_contribuinte_nome'])['id_pessoa'].nunique().to_frame().reset_index()
tot = freq_classecontrib['id_pessoa'].sum()
freq_classecontrib['perc'] = np.round(freq_classecontrib['id_pessoa']/tot, 4)
freq_classecontrib

,class_contribuinte_nome,id_pessoa,perc
0,CONTRIB BOM,4786,0.0100
1,CONTRIB EXCEL,33495,0.0700
2,CONTRIB PESSIMO,106658,0.2228
3,CONTRIB RAZOAVEL,165132,0.3449
4,CONTRIB RUIM,68060,0.1422
5,PRIMEIRA DIVIDA,100601,0.2101


# Previsão

In [234]:
matriz_previsao_class = pessoas[['situacao_cobranca', 'num_dist_cda', 'quantidade_reparcelamento', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]
matriz_previsao_class

,situacao_cobranca,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
0,2.0,1,0,1.0,1.0
1,3.0,4,0,1.0,1.0
2,3.0,1,0,1.0,1.0
3,0.0,1,0,1.0,1.0
4,0.0,2,2,1.0,1.0
...,...,...,...,...,...
494408,3.0,4,0,0.0,0.0
494409,0.0,7,0,0.0,0.0
494410,0.0,2,0,0.0,0.0
494411,0.0,8,0,0.0,0.0


In [235]:
pessoas['class_contribuinte'] = model_predict_contribuinte.predict(matriz_previsao_class)
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0,0,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,0,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,0,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA


In [236]:
pessoas.loc[pessoas['num_dist_cda'] == 1, 'class_contribuinte'] = 5
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,INACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,MUITO ACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA


# Pesos dos Contribuintes

In [237]:
pesos_analise_disc_aux = pesos_analise_disc[["coeficiente",	"class_contribuinte_nome",	"class_contribuinte"]]

In [238]:
auxx = pessoas[pessoas['class_contribuinte'] == 5]
media_num_dist_cda = auxx['situacao_cobranca'].mean()
media_num_dist_cda


1.6502006524814505

- 4	CONTRIB PESSIMO -0.79044 = pg mau + nao repar + num medio de cdas + pouquíssimo acessível
- 1	CONTRIB RUIM    -0.89564 = pg mau + nao repar + num alto cdas + pouco acessível
- 0	CONTRIB RAZOAVEL 0.84045 = pg medio + nao repar + num baixo de cdas + pouco acessível
- 3	CONTRIB BOM      0.18231 = pg bem + repar mt + num alto cdas + mt acessível 
- 2	CONTRIB EXCEL    1.52929 = pg melhor + repar bem + num medio de cdas + mt acessível 
- 5	PRIMEIRA DIVIDA  2.14338 = pg bem  + nao repar + uma CDA + pouco acessível

In [239]:
# Dando pesos para os tipos de contribuintes
pessoas.loc[pessoas['class_contribuinte'] == 4, 'class_contribuinte_peso'] = -0.89564
pessoas.loc[pessoas['class_contribuinte'] == 1, 'class_contribuinte_peso'] = -0.79044
pessoas.loc[pessoas['class_contribuinte'] == 0, 'class_contribuinte_peso'] =  0.18231
pessoas.loc[pessoas['class_contribuinte'] == 3, 'class_contribuinte_peso'] =  0.84045 
pessoas.loc[pessoas['class_contribuinte'] == 2, 'class_contribuinte_peso'] =  2.14338
pessoas.loc[pessoas['class_contribuinte'] == 5, 'class_contribuinte_peso'] =  1.52929

In [241]:
# Nomeando a classificação com label de prioridade
pessoas = pd.merge(pessoas, 
         pesos_analise_disc_aux, 
         on = ["class_contribuinte", "class_contribuinte_nome"],
         how = "left")

pessoas = pessoas.rename(columns = {'coeficiente':'class_contribuinte_peso_alt'})
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,...,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil,class_contribuinte_peso,class_contribuinte_peso,class_contribuinte_peso_alt
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,...,2.0,BEM ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,1.52929,2.14338,2.14338
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,...,3.0,MUITO ACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.18231,0.84045,0.84045
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,...,3.0,MUITO ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,1.52929,2.14338,2.14338
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,...,0.0,INACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,1.52929,2.14338,2.14338
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,...,0.0,INACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.18231,0.84045,0.84045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,...,3.0,MUITO ACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.18231,0.84045,0.84045
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,...,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA,-0.89564,-0.79044,-0.79044
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,...,0.0,INACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.18231,0.84045,0.84045
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,...,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA,-0.89564,-0.79044,-0.79044


In [242]:
pessoas = pessoas.drop(columns = {'class_contribuinte_peso'})

In [244]:
pessoas = pessoas.rename(columns = {'class_contribuinte_peso_alt':'class_contribuinte_peso'})

# Feature store do Contribuinte

In [245]:
df_feature_store_contribuinte = pessoas#.reset_index()
df_feature_store_contribuinte
# df_feature_store_contribuinte = df_feature_store_contribuinte[['id_pessoa', 'situacao', 'cpf_existe', 'edificacao', 'qtd_notas_2anos', 
#                                                                'situacao_ativa', 'status_situacao', 
#                                                                'deb_totais','deb_pagos', 'valor_tot', 'valor_pago', 
#                                                                'frequencia_da_pessoa', 'total_debitos_pessoa', 'debitos_pagos_pessoa', 'valor_total_pessoa', 'valor_pago_pessoa', 
#                                                                'historico_pagamento_em_qtd', 'historico_pagamento_em_valor', 
#                                                                'class_contribuinte', 'class_contribuinte_nome', 'class_contribuinte_peso']]

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,...,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil,class_contribuinte_peso
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,...,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,2.14338
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,...,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,...,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,2.14338
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,2.14338
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,...,1,2.0,3.0,MUITO ACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA,-0.79044
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA,-0.79044


In [246]:
df_feature_store_contribuinte.loc[df_feature_store_contribuinte['historico_pagamento_em_valor'] > 1, 'historico_pagamento_em_valor'] = 1
df_feature_store_contribuinte

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,vlr_pago,qtd_notas_2anos,edificacao,...,cpf_cnpj_existe,perfil_acessivel,situacao_cobranca,class_situacao_cobranca,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil,class_contribuinte_peso
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,...,1,1.0,2.0,BEM ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,2.14338
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,...,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,...,1,2.0,3.0,MUITO ACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,2.14338
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,1.0,1.0,5,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,2.14338
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,1.0,1.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,...,1,2.0,3.0,MUITO ACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA,-0.79044
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,0.0,0.0,0,CONTRIB RAZOAVEL,CONTROLADO,0.84045
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,...,1,0.0,0.0,INACESSÍVEL,0.0,0.0,4,CONTRIB PESSIMO,NÃO PAGA,-0.79044


In [247]:
df_feature_store_contribuinte.groupby('class_contribuinte_nome')['id_pessoa'].nunique().to_frame().reset_index()

,class_contribuinte_nome,id_pessoa
0,CONTRIB BOM,4786
1,CONTRIB EXCEL,33495
2,CONTRIB PESSIMO,106658
3,CONTRIB RAZOAVEL,165132
4,CONTRIB RUIM,68060
5,PRIMEIRA DIVIDA,100601


# Salvar no S3

In [251]:
print("Inicia a conexão com S3 para inscrição dos dados")
# Cria conexão ao s3 e preenche a tabela com os dados
s3_resource = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
    )

# Salva o arquivo em feature_store_contribuinte_2.csv
up_s3_files(dataframe=df_feature_store_contribuinte, 
            bucket_name=os.getenv("S3_BUCKET_NAME"), 
            folder_name=os.getenv("S3_FOLDER_NAME"), 
            file_name='feature_store_contribuinte_prime_2.csv')

print("Dados atualizados e persistidos no bucket S3")
print("Processo finalizado")

Inicia a conexão com S3 para inscrição dos dados
Dados atualizados e persistidos no bucket S3
Processo finalizado
